# Предсказание атака/не атака методом KNN

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [151]:
from sklearn.externals import joblib
# для сохранения обученных моделей

In [ ]:
big = pd.read_csv('big.csv', sep = '	')

In [154]:
big

,day,hour,addr,port,addr.1,port.1,enum(icmp),enum(tcp),enum(upd),string.1(irc),...,count.2,string.3,count.3,count.4,count.5,count.6,set[string](-),set[string]((empty)),string.4,string.5
0,298,12,6805,58687,369334,123,0.0,0.0,1.0,0.0,...,0,14,1,76,1,76,1.0,0.0,0,-
1,298,12,6805,1900,420753,1900,0.0,0.0,1.0,0.0,...,0,7,24,8208,0,0,1.0,0.0,0,-
2,298,12,6805,32893,369335,123,0.0,0.0,1.0,0.0,...,0,14,1,76,1,76,1.0,0.0,0,-
3,298,12,6805,53395,316235,443,0.0,1.0,0.0,0.0,...,0,126,5,212,3,144,1.0,0.0,0,-
4,298,12,6805,52801,292458,53,0.0,0.0,1.0,0.0,...,0,14,1,62,1,339,1.0,0.0,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211508,10,21,6809,123,484566,123,0.0,0.0,1.0,0.0,...,0,7,1,76,0,0,1.0,0.0,0,-
1211509,10,20,6809,51834,509175,23,0.0,1.0,0.0,0.0,...,0,103,337,18203,362,19519,1.0,0.0,1,C&C
1211510,10,22,6809,123,611845,123,0.0,0.0,1.0,0.0,...,0,7,1,76,0,0,1.0,0.0,0,-
1211511,10,22,6809,123,646453,123,0.0,0.0,1.0,0.0,...,0,7,1,76,0,0,1.0,0.0,0,-


In [ ]:
y = big['string.4']
x = big.drop('string.4', axis = 1)
x = x.drop('string.5', axis = 1)

In [75]:
x

,day,hour,addr,port,addr.1,port.1,enum(icmp),enum(tcp),enum(upd),string.1(irc),...,count.1,string.2,count.2,string.3,count.3,count.4,count.5,count.6,set[string](-),set[string]((empty))
0,298,12,6805,58687,369334,123,0.0,0.0,1.0,0.0,...,48,10,0,14,1,76,1,76,1.0,0.0
1,298,12,6805,1900,420753,1900,0.0,0.0,1.0,0.0,...,0,6,0,7,24,8208,0,0,1.0,0.0
2,298,12,6805,32893,369335,123,0.0,0.0,1.0,0.0,...,48,10,0,14,1,76,1,76,1.0,0.0
3,298,12,6805,53395,316235,443,0.0,1.0,0.0,0.0,...,0,10,0,126,5,212,3,144,1.0,0.0
4,298,12,6805,52801,292458,53,0.0,0.0,1.0,0.0,...,311,10,0,14,1,62,1,339,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211508,10,21,6809,123,484566,123,0.0,0.0,1.0,0.0,...,-1,6,0,7,1,76,0,0,1.0,0.0
1211509,10,20,6809,51834,509175,23,0.0,1.0,0.0,0.0,...,627,7,0,103,337,18203,362,19519,1.0,0.0
1211510,10,22,6809,123,611845,123,0.0,0.0,1.0,0.0,...,-1,6,0,7,1,76,0,0,1.0,0.0
1211511,10,22,6809,123,646453,123,0.0,0.0,1.0,0.0,...,-1,6,0,7,1,76,0,0,1.0,0.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=10, test_size=0.7)

In [ ]:
grid_knn = GridSearchCV(KNeighborsClassifier(),
                             param_grid={'n_neighbors': range(1,20)},
                             cv = 5)

In [79]:
%%time
grid_knn.fit(X_train, y_train)

Wall time: 31min


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None, param_grid={'n_neighbors': range(1, 20)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [82]:
accuracy_score(y_test, grid_knn.predict(X_test))

0.9876754003254486

In [89]:
grid_knn.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [92]:
big.corr()

,day,hour,addr,port,addr.1,port.1,enum(icmp),enum(tcp),enum(upd),string.1(irc),...,string.2,count.2,string.3,count.3,count.4,count.5,count.6,set[string](-),set[string]((empty)),string.4
day,1.000000,-0.042820,-0.005720,0.066334,-0.114511,-0.113001,-0.021793,0.075885,-0.070976,2.320928e-01,...,-0.278370,0.000998,-0.026428,-0.002396,-0.003075,0.004303,0.003434,0.786613,-0.786613,0.095540
hour,-0.042820,1.000000,0.001548,-0.023030,0.028616,-0.003462,0.000636,-0.005493,0.005381,1.241632e-02,...,0.032659,0.003288,0.019742,0.001217,0.001192,-0.000100,-0.000305,-0.009557,0.009557,-0.008904
addr,-0.005720,0.001548,1.000000,-0.120293,-0.000554,-0.021100,0.228895,-0.037347,-0.021576,-1.401816e-03,...,-0.135409,-0.000155,-0.031236,-0.000051,-0.000036,-0.000051,-0.000025,-0.000694,0.000694,-0.034113
port,0.066334,-0.023030,-0.120293,1.000000,-0.039611,-0.004183,-0.529576,0.323512,-0.189483,2.831947e-02,...,0.074560,0.003371,0.190771,-0.000495,-0.000499,0.000300,0.000357,0.024564,-0.024564,0.300089
addr.1,-0.114511,0.028616,-0.000554,-0.039611,1.000000,0.058990,-0.018493,-0.073359,0.078859,-1.018721e-02,...,0.086702,0.002128,-0.033328,0.001471,0.001502,-0.000935,-0.001128,-0.079673,0.079673,-0.071923
port.1,-0.113001,-0.003462,-0.021100,-0.004183,0.058990,1.000000,-0.092766,-0.576192,0.605804,-1.486183e-02,...,0.051050,-0.003198,-0.375817,-0.000985,-0.000708,-0.001519,-0.000729,-0.113683,0.113683,-0.626456
enum(icmp),-0.021793,0.000636,0.228895,-0.529576,-0.018493,-0.092766,1.000000,-0.165503,-0.091901,-4.627712e-03,...,-0.596749,-0.000546,-0.138485,-0.000176,-0.000120,-0.000254,-0.000127,-0.016696,0.016696,-0.151947
enum(tcp),0.075885,-0.005493,-0.037347,0.323512,-0.073359,-0.576192,-0.165503,1.000000,-0.966826,2.796145e-02,...,0.010179,0.003301,0.631146,-0.000689,-0.001224,0.001507,0.000764,0.009999,-0.009999,0.917963
enum(upd),-0.070976,0.005381,-0.021576,-0.189483,0.078859,0.605804,-0.091901,-0.966826,1.000000,-2.703385e-02,...,0.144285,-0.003192,-0.601395,0.000742,0.001267,-0.001456,-0.000739,-0.005772,0.005772,-0.887505
string.1(irc),0.232093,0.012416,-0.001402,0.028319,-0.010187,-0.014862,-0.004628,0.027961,-0.027034,1.000000e+00,...,0.086567,-0.000161,0.203621,0.000256,-0.000022,0.001984,0.000068,0.209268,-0.209268,0.030456


# Для улучшения модели удаляем все "лишнии" признаки

In [ ]:
x = x.drop('day', axis = 1) # и другие

In [142]:
x.head() # Оставшиеся признаки

,port,port.1,enum(icmp),enum(tcp),enum(upd),string.1(ssh),string.1(dns),string.1(-),string.2,string.3
0,58687,123,0.0,0.0,1.0,0.0,0.0,1.0,10,14
1,1900,1900,0.0,0.0,1.0,0.0,0.0,1.0,6,7
2,32893,123,0.0,0.0,1.0,0.0,0.0,1.0,10,14
3,53395,443,0.0,1.0,0.0,0.0,0.0,0.0,10,126
4,52801,53,0.0,0.0,1.0,0.0,1.0,1.0,10,14


In [138]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=10, test_size=0.7)

In [ ]:
grid_knn_1 = GridSearchCV(KNeighborsClassifier(),
                             param_grid={'n_neighbors': range(1,20)},
                             cv = 5)

In [140]:
%%time
grid_knn_1.fit(X_train, y_train)

Wall time: 18min 8s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None, param_grid={'n_neighbors': range(1, 20)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [141]:
accuracy_score(y_test, grid_knn_1.best_estimator_.predict(X_test))

0.9971181284343089

In [147]:
x.to_csv('small', sep = '	')
# сохраняем фрейм с меньшим числом признаков

In [ ]:
joblib.dump(grid_knn_1, 'grid_knn_1.pkl')
joblib.dump(grid_knn, 'grid_knn.pkl')
# сохраняем обученные модели